In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/siblose/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pasta'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "results_in_progress_NY_pasta.json"
JSON_FILE

'results_in_progress_NY_pasta.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [ ]:
#Use the function

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] results_in_progress_NY_pasta.json not found. Saving empty list to new file.
- 0 previous results found.


515

In [7]:
#read only first 1000 pages- api limit
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/515 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [ ]:
#Convert .json to dataframe
#Load in the "results in progress" JSON file into a DataFrame:

In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4888,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3192,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
2,ePJlqiIhoVKIJW69IE53rg,botte-brooklyn-brooklyn,Botte Brooklyn,https://s3-media1.fl.yelpcdn.com/bphoto/gytaiw...,False,https://www.yelp.com/biz/botte-brooklyn-brookl...,79,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.69446, 'longitude': -73.98316}","[pickup, delivery]",NaN,"{'address1': '327 Gold St', 'address2': None, ...",+17185865090,(718) 586-5090,1546.804489
3,yXMZSuLJEpNu24yhKey9XA,forma-pasta-factory-brooklyn-3,Forma Pasta Factory,https://s3-media1.fl.yelpcdn.com/bphoto/6LEtDQ...,False,https://www.yelp.com/biz/forma-pasta-factory-b...,128,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.686023409875474, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '5 Greene Ave', 'address2': None,...",+13477991333,(347) 799-1333,2800.607052
4,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1424,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.718554, 'longitude': -73.997447}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,aPn2_EtjfHduXEmEBNxuFA,rosas-pizza-brooklyn,Rosa's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/TtcGEK...,False,https://www.yelp.com/biz/rosas-pizza-brooklyn?...,196,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.7139499, 'longitude': -73.95563}","[delivery, pickup]",$,"{'address1': '374 Metropolitan Ave', 'address2...",+17189632378,(718) 963-2378,3387.060216
996,Om6KGbN524K6pu9IkPT0Rg,maite-brooklyn,Maite,https://s3-media1.fl.yelpcdn.com/bphoto/dmJfOJ...,False,https://www.yelp.com/biz/maite-brooklyn?adjust...,219,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.0,"{'latitude': 40.69924, 'longitude': -73.926797}",[delivery],$$,"{'address1': '159 Central Ave', 'address2': ''...",+17183663090,(718) 366-3090,5731.450254
997,jjLcqGI11ThxAhcftEh1Aw,fraunces-tavern-new-york-6,Fraunces Tavern,https://s3-media3.fl.yelpcdn.com/bphoto/CFeN_8...,False,https://www.yelp.com/biz/fraunces-tavern-new-y...,1128,"[{'alias': 'tradamerican', 'title': 'American'...",3.5,"{'latitude': 40.703419116685254, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '54 Pearl St', 'address2': '', 'a...",+12129681776,(212) 968-1776,1456.524312
998,auKeIP4_nJuWkBIUcDObvA,wahizza-ridgefield-park,Wahizza,https://s3-media4.fl.yelpcdn.com/bphoto/YUF7D1...,False,https://www.yelp.com/biz/wahizza-ridgefield-pa...,27,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.857423251606264, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '213 Main St', 'address2': None, ...",+12018706059,(201) 870-6059,17092.747276
999,NQ6CE2KiS1e-yMNgI5OTpw,sandros-italian-manhasset-5,Sandro's Italian,https://s3-media3.fl.yelpcdn.com/bphoto/_HiV8g...,False,https://www.yelp.com/biz/sandros-italian-manha...,62,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 40.79337, 'longitude': -73.69131}",[],$$$,"{'address1': '1496 Northern Blvd', 'address2':...",+15164674266,(516) 467-4266,27351.910196


In [10]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

122

In [ ]:
#If it was not alist we were going to do the below, but not in this case
#final_df.duplicated().sum()

In [11]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [14]:
# save the final results to a compressed csv
final_df.to_csv('final_results_NY_pasta.csv', compression='gzip',index=False)